In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"cindyz3399","key":"a90282dd0cc53046facead259a1a8adf"}'}

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c human-protein-atlas-image-classification

100% 17.5G/17.5G [03:11<00:00, 135MB/s]
100% 17.5G/17.5G [03:11<00:00, 98.0MB/s]


In [6]:
!unzip human-protein-atlas-image-classification.zip -d /content/data/

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/train/f5b629aa-bbbb-11e8-b2ba-ac1f6b6435d0_blue.png  
  inflating: /content/data/train/f5b629aa-bbbb-11e8-b2ba-ac1f6b6435d0_green.png  
  inflating: /content/data/train/f5b629aa-bbbb-11e8-b2ba-ac1f6b6435d0_red.png  
  inflating: /content/data/train/f5b629aa-bbbb-11e8-b2ba-ac1f6b6435d0_yellow.png  
  inflating: /content/data/train/f5b6f5c0-bbc4-11e8-b2bc-ac1f6b6435d0_blue.png  
  inflating: /content/data/train/f5b6f5c0-bbc4-11e8-b2bc-ac1f6b6435d0_green.png  
  inflating: /content/data/train/f5b6f5c0-bbc4-11e8-b2bc-ac1f6b6435d0_red.png  
  inflating: /content/data/train/f5b6f5c0-bbc4-11e8-b2bc-ac1f6b6435d0_yellow.png  
  inflating: /content/data/train/f5baae8e-bbc1-11e8-b2bb-ac1f6b6435d0_blue.png  
  inflating: /content/data/train/f5baae8e-bbc1-11e8-b2bb-ac1f6b6435d0_green.png  
  inflating: /content/data/train/f5baae8e-bbc1-11e8-b2bb-ac1f6b6435d0_red.png  
  inflating: /content/data/train/f5baae8e-bbc1-11e8-b2

In [7]:
import os

data_dir = "/content/data/"
print("File Name:", os.listdir(data_dir))

File Name: ['sample_submission.csv', 'train.csv', 'test', 'train']


In [8]:
# Check train
print("Train:", len(os.listdir(os.path.join(data_dir, "train"))))

# Check test
print("Test:", len(os.listdir(os.path.join(data_dir, "test"))))

Train: 124288
Test: 46808


In [9]:
import pandas as pd

# Train CSV
train_csv_path = os.path.join(data_dir, "train.csv")
train_labels = pd.read_csv(train_csv_path)

# Check
print(train_labels.head())

                                     Id   Target
0  00070df0-bbc3-11e8-b2bc-ac1f6b6435d0     16 0
1  000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  7 1 2 0
2  000a9596-bbc4-11e8-b2bc-ac1f6b6435d0        5
3  000c99ba-bba4-11e8-b2b9-ac1f6b6435d0        1
4  001838f8-bbca-11e8-b2bc-ac1f6b6435d0       18


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from PIL import Image
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:


NUM_CLASSES = 28

# One-hot
def multi_label_encoding(labels):
    encoded = np.zeros(NUM_CLASSES, dtype=np.float32)
    for label in labels.split():
        encoded[int(label)] = 1
    return encoded

train_labels['Encoded_Labels'] = train_labels['Target'].apply(multi_label_encoding)

print(train_labels.head())

                                     Id   Target  \
0  00070df0-bbc3-11e8-b2bc-ac1f6b6435d0     16 0   
1  000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  7 1 2 0   
2  000a9596-bbc4-11e8-b2bc-ac1f6b6435d0        5   
3  000c99ba-bba4-11e8-b2b9-ac1f6b6435d0        1   
4  001838f8-bbca-11e8-b2bc-ac1f6b6435d0       18   

                                      Encoded_Labels  
0  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1  [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...  
3  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  


In [12]:
label_names = {
    0:  "Nucleoplasm",
    1:  "Nuclear membrane",
    2:  "Nucleoli",
    3:  "Nucleoli fibrillar center",
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",
    6:  "Endoplasmic reticulum",
    7:  "Golgi apparatus",
    8:  "Peroxisomes",
    9:  "Endosomes",
    10:  "Lysosomes",
    11:  "Intermediate filaments",
    12:  "Actin filaments",
    13:  "Focal adhesion sites",
    14:  "Microtubules",
    15:  "Microtubule ends",
    16:  "Cytokinetic bridge",
    17:  "Mitotic spindle",
    18:  "Microtubule organizing center",
    19:  "Centrosome",
    20:  "Lipid droplets",
    21:  "Plasma membrane",
    22:  "Cell junctions",
    23:  "Mitochondria",
    24:  "Aggresome",
    25:  "Cytosol",
    26:  "Cytoplasmic bodies",
    27:  "Rods & rings"
}

In [13]:

def fill_targets(row):
    # Convert space-separated string to an array of integers
    target_nums = np.array(row.Target.split(" ")).astype(int)
    for num in target_nums:
        name = label_names[num]
        row.loc[name] = 1
    return row

In [14]:

df_train = train_labels.apply(fill_targets, axis=1).fillna(0)
df_train.head()

,Actin filaments,Aggresome,Cell junctions,Centrosome,Cytokinetic bridge,Cytoplasmic bodies,Cytosol,Encoded_Labels,Endoplasmic reticulum,Endosomes,...,Nuclear bodies,Nuclear membrane,Nuclear speckles,Nucleoli,Nucleoli fibrillar center,Nucleoplasm,Peroxisomes,Plasma membrane,Rods & rings,Target
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,16 0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,7 1 2 0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
protein_count=df_train.Target.value_counts()

target_counts = df_train.drop(["Id", "Target"],axis=1).sum(axis=0).sort_values(ascending=False)
plt.figure(figsize=(12,8))
sns.barplot(y=target_counts.index.values, x=target_counts.values, order=target_counts.index, palette="viridis")
plt.show()


# I'm confused about this part since the ratio of each sort added up less than 1(IT suppose to be larger than one since overlap existing)
# I tried to do stratify but since there are multilable so I gave up. I enlarge the scale of train dataloader and used the left part of train dataset as test dataset.
# The original test dataset file don't have lable.

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [16]:
import torchvision.transforms as transforms
from PIL import Image

# define picture preprocessing
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # same size
    transforms.RandomHorizontalFlip(),  # random flipping
    transforms.RandomRotation(15),  # rotat
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),  # Convert to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # normalization
])

data_dir = "/content/data/"
train_dir = os.path.join(data_dir, "train")

# Get a list of all image files in the train directory
image_files = [f for f in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, f))]

# Check if there are any image files in the directory
if image_files:
    # Use the first image file in the list
    img_path = os.path.join(train_dir, image_files[0])
    image = Image.open(img_path).convert("RGB")
    image = train_transform(image)
    print(image.shape)
else:
    print("No image files found in the train directory.")


torch.Size([3, 224, 224])


In [17]:
# load training data
data_dir = "/content/data"
train_csv_path = os.path.join(data_dir, "train.csv")
train_images_path = os.path.join(data_dir, "train")

# read CSV
train_labels = pd.read_csv(train_csv_path)
print("training data:", len(train_labels))
print("first lines:\n", train_labels.head())

# Target string type
train_labels["Target"] = train_labels["Target"].astype(str)

training data: 31072
first lines:
                                      Id   Target
0  00070df0-bbc3-11e8-b2bc-ac1f6b6435d0     16 0
1  000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  7 1 2 0
2  000a9596-bbc4-11e8-b2bc-ac1f6b6435d0        5
3  000c99ba-bba4-11e8-b2b9-ac1f6b6435d0        1
4  001838f8-bbca-11e8-b2bc-ac1f6b6435d0       18


In [18]:
class ProteinDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_id = self.dataframe.iloc[idx]["Id"]
        labels = self.dataframe.iloc[idx]["Target"]

        # read R/G/B/Y images
        img_red = Image.open(os.path.join(self.img_dir, f"{img_id}_red.png"))
        img_green = Image.open(os.path.join(self.img_dir, f"{img_id}_green.png"))
        img_blue = Image.open(os.path.join(self.img_dir, f"{img_id}_blue.png"))
        img_yellow = Image.open(os.path.join(self.img_dir, f"{img_id}_yellow.png")).convert("L")

        # Convert to tensors
        img_red = transforms.ToTensor()(img_red)
        img_green = transforms.ToTensor()(img_green)
        img_blue = transforms.ToTensor()(img_blue)
        img_yellow = transforms.ToTensor()(img_yellow)

        # Merge into 4 channels
        image = torch.cat([img_red, img_green, img_blue, img_yellow], dim=0)

        # transfer Target to One-hot encoding
        NUM_CLASSES = 28
        encoded_labels = torch.zeros(NUM_CLASSES, dtype=torch.float32)
        for label in str(labels).split():
            encoded_labels[int(label)] = 1

        if self.transform:
            image = self.transform(image)

        return image, encoded_labels

In [21]:



# Defining Data Transformations (Normalize & Resize)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.ToTensor(),  # Remove this line as it's already done in __getitem__
    transforms.Normalize(mean=[0.5, 0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5, 0.5])
])


# Create a dataset
train_dataset = ProteinDataset(train_labels, train_images_path, transform=train_transform)

# Pick indices for 2/3 and remaining 1/3
subset_size = 2 * (len(train_dataset) // 3)
all_indices = np.arange(len(train_dataset))
train_indices = np.random.choice(all_indices, size=subset_size, replace=False)
validation_indices = np.setdiff1d(all_indices, train_indices)

# Use 'SubsetRandomSampler' for train and validate
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, sampler=SubsetRandomSampler(train_indices))
validation_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, sampler=SubsetRandomSampler(validation_indices))




In [22]:
# Load DenseNet121 train the model
model = models.densenet121(pretrained=True)

# Modify the first layer to support 4-channel inputs
model.features.conv0 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modifying the Last Layer Classifier (Class 28)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 28)

# Send to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 97.4MB/s]


In [23]:
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_loader):.4f}")

print("Successful！")

KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

    print(f"Test Loss: {total_loss/len(dataloader):.4f}")

evaluate_model(model, train_loader)

##Confusion Matrix

In [ ]:
import torch
from sklearn.metrics import multilabel_confusion_matrix, classification_report
import numpy as np
from torch.utils.data import DataLoader


model.eval()
y_true, y_pred = [], []

for images, labels in validation_loader:
    images, labels = images.to(device), labels.to(device)

    with torch.no_grad():
        outputs = torch.sigmoid(model(images))
        preds = (outputs > 0.5).cpu().numpy()

    labels = labels.cpu().numpy()


    y_true.extend(labels)
    y_pred.extend(preds)


y_true = np.array(y_true)
y_pred = np.array(y_pred)


cm = multilabel_confusion_matrix(y_true, y_pred)
print("Multilabel Confusion Matrix:\n", cm)

report = classification_report(y_true, y_pred, zero_division=0)
print(report)


In [ ]:
def predict(model, img_id, train_images_path="/content/data/train"):
    model.eval()

    colors = ["red", "green", "blue", "yellow"]
    img_tensors = []

    for color in colors:
        img_path = os.path.join(train_images_path, f"{img_id}_{color}.png")
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Not found: {img_path}")

        if color == "yellow":
            img = Image.open(img_path).convert("L")
        else:
            img = Image.open(img_path)

        transform_pipeline = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])

        img_tensor = transform_pipeline(img)
        img_tensors.append(img_tensor)

    # Merge 4 channels
    image = torch.cat(img_tensors, dim=0).unsqueeze(0).to(device)

    # Predict
    with torch.no_grad():
        output = model(image)
        output = torch.sigmoid(output).cpu().numpy()

    predicted_labels = (output[0] > 0.5).astype(int)
    return predicted_labels

# Predictions are made using samples that do exist within the folder
sample_img_id = os.listdir(train_images_path)[0].split("_")[0]
predicted_labels = predict(model, sample_img_id)
print(f"Predicted Labels: {predicted_labels}")